# Live Code 2

```txt
Adnan Rio
Batch 14
```

## Description


### Problem Statement

```
### Problem 1 ###
Berikut tabel yang berisikan frekuensi kata yang muncul pada kedua kalimat:

Kata	Kalimat 1	Kalimat 2
me	2	2
Jane	0	1
Julie	1	1
Linda	1	0
likes	0	1
loves	2	1
more	1	1
than	1	1

Buatlah vektor yang merupakan representasi masing-masing kalimat berdasarkan tabel di atas dan hitung cosine similarity antar kedua vektor.

### Problem 2 ###
1. Gunakan tabel `states` pada dataset `geo_us_boundaries`.
2. Buatlah query dengan kriteria sebagai berikut:
   - Pilih semua kolom dan **HANYA** data negara bagian `Colorado` (Informasi nama negara bagian terdapat di kolom `state_name`).
3. Simpan dataset dalam bentuk csv, dengan nama `h8dsft_P0LC2_<nama-students>.csv`.
4. Salin query yang telah dibuat di Google Cloud Platform, tulislah pada bagian atas notebook!
5. Tampilkan data pada notebook (Data hanya mengandung satu baris saja)!

Menggunakan konsep integral untuk menghitung luas dibawah kurva, tentukan luas negara bagian Colorado jika diketahui koordinat batas-batasnya pada data yang sudah kamu ambil dari big query. 

```

## Working Area


3.  Import Pustaka

In [58]:
#Problem 1
import pandas as pd
import numpy as np
#Problem 2
import re
from scipy import integrate 
from scipy.integrate import trapz
data=pd.read_csv('h8dsft_P0LC2_adnan_rio.csv')
data

,geo_id,region_code,division_code,state_fips_code,state_gnis_code,state,state_name,lsad_code,mtfcc_feature_class_code,functional_status,area_land_meters,area_water_meters,int_point_lat,int_point_lon,int_point_geom,state_geom
0,8,4,8,8,1779779,CO,Colorado,0,G4000,A,268419875371,1184637800,38.993848,-105.508317,POINT(-105.5083165 38.9938482),"POLYGON((-109.050949 40.222647, -109.050952 40..."


4. Data Loading

In [49]:
# Problem 1
# Membuat vektor kalimat 1 dan kalimat 2
k_1=np.array([2, 0, 1, 1, 0, 2, 1, 1])
k_2=np.array([2, 1, 1, 0, 1, 1, 1, 1])
print(k_1)
print(k_2)

[2 0 1 1 0 2 1 1]
[2 1 1 0 1 1 1 1]


In [47]:
# Problem 2
# query GCP
## SELECT *
## ROM `bigquery-public-data.geo_us_boundaries.states` AS stat
## WHERE stat.state_name ='Colorado';

def data_preprocess(data):
   long = []
   lat = []
   for row in data['state_geom'][0].split(', '):
     row = re.sub('MULTIPOLYGON','',row)
     row = re.sub('POLYGON','',row)
     row = re.sub(r'\(','',row)
     row = re.sub(r'\)','',row)
     long.append(float(row.split(' ')[0]))
     lat.append(float(row.split(' ')[1]))
   return pd.DataFrame({'long':long,'lat':lat}).sort_values('long',ascending=True)

In [51]:
data_colorado = data_preprocess(data)

In [57]:
data_area_sorted = data_for_area.sort_values('long')
data_area_sorted.head(5) #Menampilkan 5 baris pertama

,long,lat
759,-109.060204,38.278238
760,-109.060187,38.275528
751,-109.060185,38.318670
743,-109.060180,38.344908
659,-109.060176,38.462300


5.  Mathematical Calculations

In [ ]:
# Problem 1
# Menghitung cosine similarity
similarity_kon = k_1.dot(k_2)/(np.linalg.norm(k_1) * np.linalg.norm(k_2))
print(similarity_kon)

In [53]:
# Problem 2
max_lat = data_area_sorted['lat'].max() #Menentukan nilai maksimum latitude Colorado
min_lat = data_area_sorted['lat'].min() #Menentukan nilai minimum latitude Colorado
mid_lat = (max_lat+min_lat)/2 #Menentukan nilai tengah latitude Colorado

In [55]:
#Colorado dan Northern Colorado berdasarkan nilai tengah latitudenya
northern_area = data_area_sorted[data_area_sorted['lat'] >= mid_lat]
southern_area = data_area_sorted[data_area_sorted['lat'] < mid_lat]

In [ ]:
area_N = integrate.trapz(northern_area['lat'],northern_area['long']) #Menghitung area dari latitude y=0 ke latitude pada northern area
area_S = integrate.trapz(southern_area['lat'],southern_area['long']) #Menghitung area dari latitude y=0 ke latitude pada southern area
total_area = area_N-area_S #total area Colorado
print(f'Colorada sebesar {round(total_area)} derajat kuadrat.')

6.  Hasil

####Problem 1####

a. Apakah kedua kalimat memiliki konteks yang serupa? jika iya, mengapa dan jika tidak, mengapa (jawab berdasarkan hasil perhitungan cosine similaritynya)==
   iya , karena kedua kalimat memiliki cosine similaritynya 0.8215838362577491 yang mendekati 1

b. Jika meninjau dua buah vektor dan dihitung cosine similaritynya (cos theta), jelaskan secara singkat, jelas, padat apa makna cosine similarity yang      bernilai 0 dan 1 (tinjau dari posisi dua vektor di koordinat kartesian)==
   Jika semakin mendekati 1 berarti dua vektor memiliki orientasi mendekati sama. Nilai yang mendekati 0 menunjukkan bahwa kedua kalimat tersebut memiliki kemiripan yang lebih kecil.

c. Apa kekurangan dari perhitungan kemiripan menggunakan cosine similarity==
   Besarnya vektor tidak diperhitungkan, hanya arahnya artinya perbedaan nilai tidak sepenuhnya diperhitungkan atau cosine similarity tidak memperhitungkan perbedaan skala penilaian antara pengguna yang berbeda.

####Problem 2####

a. Integral jenis apa yang diterapkan pada kasus ini? (Tertentu/Tak tentu) dan mengapa demikian? berikan alasan!== tidak tentu karena luas wilayah mempunyai bentuk yang tidak beraturan akan sangat sulit untuk menentukan luasnya dikarenakan tidak ada rumus baku untuk menentukan luasnya

b. Untuk kasus ini, metode apa yang kamu pakai dalam perhitungan integral? secara simbolik atau numerik, dan mengapa? == numeric karena hanya memerlukan titik – titik koordinat , di mana menyatakan panjang dan lebar dari sebuah segiempat dengan menyatakan jumlahnya ke empat pinggiran yang berbentuk segiempat


c. Library Scipy menyediakan banyak function/modul untuk menghitung integral. Function apa yang kamu gunakan dan berikan alasan yang logis!
   Function trapz karena mencari total luas dari tiap sepanjang sumbu yang diberikan menggunakan aturan trapesium komposit.
